### Домашнее задание №8 по теме «Деревья решений»

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
pd.options.display.float_format = '{:.3f}'.format

#### Задание  
____
* Цель: изучить применение дерева решений в рамках задачи регрессии
* Описание задания: В домашнем задании нужно решить задачу регрессии. В качестве датасета необходимо взять данные о недвижимости Калифорнии из библиотеки sklearn.datasets. Целевая переменная – MedHouseVal. Прочитать информацию о признаках датасета можно, выполнив следующий код – print(fetch_california_housing().DESCR). На полученных данных построить модель регрессии и дерево решений.
* Этапы работы:

1. Получите данные и загрузите их в рабочую среду.  
2. Проведите первичный анализ.  
    a. Проверьте данные на пропуски. Удалите в случае обнаружения.  
    b. \*Нормализуйте один из признаков.  
3. Разделите выборку на обучающее и тестовое подмножества. 80% данных оставить на обучающее множество, 20% - на тестовое.  
4. Обучите модель регрессии на обучающем множестве.  
5. Для тестового множества предскажите целевую переменную и сравните с истинным значением, посчитав точность предсказания модели. Для этого используйте встроенную функцию `score`. 
6. Обучите дерево решений на обучающем множестве.  
    a. Повторите п. 5 для полученной модели.  
    b. Визуализируйте часть дерева решений. Убедитесь, что график получился читабельным. Посмотрите примеры визуализации по ссылке.  
7. Оптимизируйте глубину дерева (max_depth). \*Оптимизируйте ещё один параметр модели на выбор.  
    a. Повторите п. 5 для полученной модели.  
8. Сформулируйте выводы по проделанной работе.  
    a. Сравните точность двух моделей.  
    b. Напишите свое мнение, для каких задач предпочтительнее использовать обученные в работе модели? Какие у них есть плюсы и минусы?  
____
Для получения зачета по этому домашнему заданию, должно быть как минимум реализовано обучение двух моделей, выведена их точность, оптимизирован один параметр дерева решений.
Результат: получены знания по работе с деревом решений
Форма выполнения:

#### 1. Загружаем данные

In [2]:
california_housing = fetch_california_housing(as_frame=True)

In [3]:
print(fetch_california_housing().DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [4]:
df = california_housing.frame

In [5]:
df.head(7)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.325,41.000,6.984,1.024,322.000,2.556,37.880,-122.230,4.526
1,8.301,21.000,6.238,0.972,2401.000,2.110,37.860,-122.220,3.585
2,7.257,52.000,8.288,1.073,496.000,2.802,37.850,-122.240,3.521
3,5.643,52.000,5.817,1.073,558.000,2.548,37.850,-122.250,3.413
4,3.846,52.000,6.282,1.081,565.000,2.181,37.850,-122.250,3.422
5,4.037,52.000,4.762,1.104,413.000,2.140,37.850,-122.250,2.697
6,3.659,52.000,4.932,0.951,1094.000,2.128,37.840,-122.250,2.992


#### 2. Пропуски

In [6]:
pd.concat([df.isnull().sum().rename('cnt'), round((df.isna().mean() * 100),2).rename('%')], axis=1)

,cnt,%
MedInc,0,0.000
HouseAge,0,0.000
AveRooms,0,0.000
AveBedrms,0,0.000
Population,0,0.000
AveOccup,0,0.000
Latitude,0,0.000
Longitude,0,0.000
MedHouseVal,0,0.000


Пропусков нет. Но это и было указано в описании датасета.

Нормализация  
$$ x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}} $$

In [7]:
def get_normalize(df, cols):
    df = df.copy()
    for each in cols:
        arr = np.array(df[each])
        df[each] = np.divide((arr - arr.min()),(arr.max() - arr.min()))
    return df

# def get_normalize(df, cols):
#     df = df.copy()
#     for each in cols:
#         arr = np.array(df[each])
#         normalizer = MinMaxScaler()
#         df[each] = normalizer.fit_transform(arr.reshape(-1, 1))
#     return df

In [8]:
def get_score(df, target, random_seed=42, model=None, title='Baseline'):
    if model is None:
        model = LinearRegression()
    
    y = df[target]
    X = df.drop([target], axis = 1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed) 

    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    res_train = model.score(X_train, y_train)
    res_test = model.score(X_test, y_test)
    metrix = {}
    metrix['score_train'] = res_train
    metrix['score_test'] = res_test
    res = pd.DataFrame.from_dict(metrix, orient='index', columns=[title])
    return res

In [9]:
df = get_normalize(df, ['MedInc'])

In [10]:
get_score(df, 'MedHouseVal')

,Baseline
score_train,0.613
score_test,0.576
